In [ ]:
import os
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

# Function to load images from directory
def load_images(directory):
    images = []
    labels = []
    
    for category in os.listdir(directory):
        for filename in tqdm(os.listdir(os.path.join(directory, category))):
            image_path = os.path.join(directory, category, filename)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))
            images.append(image)
            labels.append(category)
    
    images = np.array(images, dtype='float32')
    return images, labels

# Load training and testing data
train_dir = "/train/"
test_dir = "/test/"
X_train, y_train = load_images(train_dir)
X_test, y_test = load_images(test_dir)

# Normalize pixel values
X_train = X_train / 255.
X_test = X_test / 255.

# Encode labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#  CNN model
model = Sequential([
    Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(pool_size=2),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    BatchNormalization(),
    Dropout(0.25),
    Flatten(),
    Dense(units=128, activation='relu'),
    BatchNormalization(),
    Dropout(0.25),
    Dense(units=4, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, mode='max', verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', patience=5, mode='max', verbose=1, factor=0.1, min_lr=0.001)
checkpoint = ModelCheckpoint(filepath='checkpoint/', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')

# Train the model
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Save the model
model.save(filepath='drowsiness_detector.h5')
